In [1]:
import re
import os
import numpy as np
import scipy.interpolate as interp
from scipy.ndimage import map_coordinates
import glob

from robotcar.python.camera_model import CameraModel
from robotcar.python.transform import build_se3_transform
from robotcar.python.image import load_image
from robotcar.python.interpolate_poses import interpolate_vo_poses, interpolate_ins_poses
from robotcar.python.build_pointcloud import build_pointcloud

from monodepth.utils.evaluation_utils import compute_errors

from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import shutil

import cv2

In [2]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

# Configuration

### Constants

In [27]:
img_index_start = 1000
img_index_stop = 32000
img_index_step = 100

# micro!!s history for pointcloud generation
time_minus = 8000000
time_plus = 0

# Width and height of ground truth
height = 960
width = 1280

# Model Maximum and Minimum Prediction Depth (m) --> Taken from Monodepth Paper
pred_max = 80
pred_min = 1e-3

### Directories

In [9]:
image_dir = '/media/psf/Data/Downloads/data_test/2014-12-05-11-09-10/2014-12-05-11-09-10_stereo_left/2014-12-05-11-09-10/stereo/left/'
extrinsics_dir = 'robotcar/extrinsics/'
models_dir = 'robotcar/models/'
lidar_dir = '/media/psf/Data/Downloads/data_test/2014-12-05-11-09-10/2014-12-05-11-09-10_ldmrs/ldmrs/'

lidar = 'ldmrs'

vo_poses_file = '/media/psf/Data/Downloads/data_test/2014-12-05-11-09-10/2014-12-05-11-09-10_vo/vo/vo.csv'
ins_poses_file = '/media/psf/Data/Downloads/data_test/2014-12-05-11-09-10/2014-12-05-11-09-10_gps/gps/ins.csv'

test_image_dir = 'test_images'

In [10]:
def convert_disps_to_depths(pred_disps, height, width, pred_max, pred_min):
    pred_depths = np.zeros((pred_disps.shape[0],height,width))
    for i in range(pred_disps.shape[0]):
        # Resize to ground truth shape
        pred_disp = width * cv2.resize(pred_disps[i,:,:], (width, height), interpolation=cv2.INTER_LINEAR)
        # Convert to depth
        pred_depths[i,:,:] = 983.044006 * 0.24 / pred_disp
    
    # Set 0 for values out of range
    # TODO check effect if set to max or min like paper do
    pred_depths[((pred_depths > pred_max) | (pred_depths < pred_min))] = 0
    
    return pred_depths

### Camera and LIDAR Extrinsics

In [11]:
model = CameraModel(models_dir, image_dir)

extrinsics_path = os.path.join(extrinsics_dir, model.camera + '.txt')
with open(extrinsics_path) as extrinsics_file:
    extrinsics = [float(x) for x in next(extrinsics_file).split(' ')]

G_camera_vehicle = build_se3_transform(extrinsics)

with open(os.path.join(extrinsics_dir, lidar + '.txt')) as extrinsics_file:
    extrinsics = next(extrinsics_file)
G_posesource_laser = build_se3_transform([float(x) for x in extrinsics.split(' ')])

# [RobotCar] Preprocess Images and Save to test folder

In [18]:
# Create/clear test_images folder and create filenames file
if os.path.exists(test_image_dir):
    shutil.rmtree(test_image_dir) 
os.makedirs(test_image_dir)

with open(test_image_dir + '/test_images.txt', "a+") as filenames_file:
    
    c = 1
    for img_index in range(img_index_start, img_index_stop, img_index_step):
        print("Image {} from {}".format(c, np.round((img_index_stop-img_index_start)/img_index_step)))
        c = c + 1
        timestamps_path = os.path.join(image_dir, os.pardir, model.camera + '.timestamps')
        if not os.path.isfile(timestamps_path):
            timestamps_path = os.path.join(image_dir, os.pardir, os.pardir, model.camera + '.timestamps')

        with open(timestamps_path) as timestamps_file:
            for i, line in enumerate(timestamps_file):
                if i == img_index:
                    timestamp_img = int(line.split(' ')[0])

        image_path = os.path.join(image_dir, str(timestamp_img) + '.png')
        image = load_image(image_path, model)
        img = Image.fromarray(image)
        img.save('test_images/' + str(timestamp_img) + '.png')

        # Append to filenames file
        if img_index == img_index_start:
            filenames_file.write(str(timestamp_img) + '.png')
        else:
            filenames_file.write('\n' + str(timestamp_img) + '.png')
filenames_file.close()

Image 1 from 310
Image 2 from 310
Image 3 from 310
Image 4 from 310
Image 5 from 310
Image 6 from 310
Image 7 from 310
Image 8 from 310
Image 9 from 310
Image 10 from 310
Image 11 from 310
Image 12 from 310
Image 13 from 310
Image 14 from 310
Image 15 from 310
Image 16 from 310
Image 17 from 310
Image 18 from 310
Image 19 from 310
Image 20 from 310
Image 21 from 310
Image 22 from 310
Image 23 from 310
Image 24 from 310
Image 25 from 310
Image 26 from 310
Image 27 from 310
Image 28 from 310
Image 29 from 310
Image 30 from 310
Image 31 from 310
Image 32 from 310
Image 33 from 310
Image 34 from 310
Image 35 from 310
Image 36 from 310
Image 37 from 310
Image 38 from 310
Image 39 from 310
Image 40 from 310
Image 41 from 310
Image 42 from 310
Image 43 from 310
Image 44 from 310
Image 45 from 310
Image 46 from 310
Image 47 from 310
Image 48 from 310
Image 49 from 310
Image 50 from 310
Image 51 from 310
Image 52 from 310
Image 53 from 310
Image 54 from 310
Image 55 from 310
Image 56 from 310
I

# [Monodepth] Get Predictions from Model


In [22]:
!python monodepth/monodepth_main.py --mode test --data_path test_images/ --filenames_file test_images/test_images.txt --log_directory log/ --checkpoint_path monodepth/model_kitti_ft_01/model-1900.ckpt --output_directory test_images/


now testing 310 files
step 0 from 310
step 1 from 310
^C


# [Monodepth] Convert predicted disparities to depth

In [23]:
pred_disparities_file = 'test_images/disparities_ft.npy'

In [24]:
pred_disparities = np.load(pred_disparities_file)
pred_depths = convert_disps_to_depths(pred_disparities, height, width, pred_max, pred_min)

# Convert to dictionary
pred_depths_dict = {}
i = 0
# Loop trough test files
for img_file in sorted(glob.glob1(test_image_dir,"*.png")):
    pred_depths_dict[img_file] = pred_depths[i,:,:]
    i = i + 1

# [RobotCar] Process Pointclouds and save ground truth

In [ ]:
# Initialize array
filecount = len(glob.glob1(test_image_dir,"*.png"))
# Dictionary
# Filename => Height x Width
# VO
# INS
# Local

vo_gt_depths = {}
ins_gt_depths = {}
local_gt_depths = {}
    

# Loop trough test files
for img_file in glob.glob1(test_image_dir,"*.png"):
    print("Processing:",img_file)
    timestamp_img = int(img_file.split('.')[0])

    '''
    # VO Pointcloud
    G_camera_posesource = G_camera_vehicle
    
    pointcloud, reflectance = build_pointcloud(lidar_dir, vo_poses_file, extrinsics_dir,
                                           timestamp_img - time_minus, timestamp_img + time_plus, timestamp_img)
    
    pointcloud = np.dot(G_camera_posesource, pointcloud)
    uv, depth = model.project(pointcloud, (height, width))
    uv = np.array(uv)
    depth = np.array(depth)
    mask = ((uv[0,:] < (width-1)) & (uv[1,:] < (height-1)) & (depth < pred_max) & (depth > pred_min))
    depth = depth[mask]
    uv = uv[:, mask]
    gt_img = np.zeros((height, width))
    coords = uv.transpose().astype(int)
    gt_img[coords[:,1],coords[:,0]] = depth    
    vo_gt_depths[img_file] = gt_img
    '''
    
    # INS Pointcloud
    with open(os.path.join(extrinsics_dir, 'ins.txt')) as extrinsics_file:
        extrinsics = next(extrinsics_file)
        G_camera_posesource = G_camera_vehicle * build_se3_transform([float(x) for x in extrinsics.split(' ')])

    pointcloud, reflectance = build_pointcloud(lidar_dir, ins_poses_file, extrinsics_dir,
                                           timestamp_img - time_minus, timestamp_img + time_plus, timestamp_img)
    pointcloud = np.dot(G_camera_posesource, pointcloud)       
    uv, depth = model.project(pointcloud, (height, width))
    uv = np.array(uv)
    depth = np.array(depth)
    mask = ((uv[0,:] < (width-1)) & (uv[1,:] < (height-1)) & (depth < pred_max) & (depth > pred_min))
    depth = depth[mask]
    uv = uv[:, mask]
    gt_img = np.zeros((height, width))
    coords = uv.transpose().astype(int)
    gt_img[coords[:,1],coords[:,0]] = depth    
    ins_gt_depths[img_file] = gt_img    
    
    # Sparse Local Pointcloud
    # Only takes one lidar scan closest to when image was taken
    timestamps_path = os.path.join(lidar_dir, os.pardir, lidar + '.timestamps')
    with open(timestamps_path) as timestamps_file:
        for line in timestamps_file:
            this_timestamp = int(line.split(' ')[0])
            if this_timestamp >= timestamp_img:
                timestamp_lidar = this_timestamp
                break
    with open(os.path.join(extrinsics_dir, lidar + '.txt')) as extrinsics_file:
        extrinsics = next(extrinsics_file)
    G_posesource_laser = build_se3_transform([float(x) for x in extrinsics.split(' ')])
    local_pointcloud = np.array([[0], [0], [0], [0]])
    scan_path = os.path.join(lidar_dir, str(timestamp_lidar) + '.bin')
    scan_file = open(scan_path)
    scan = np.fromfile(scan_file, np.double)
    scan_file.close()
    scan = scan.reshape((len(scan) // 3, 3)).transpose()
    scan = np.dot(G_posesource_laser, np.vstack([scan, np.ones((1, scan.shape[1]))]))
    local_pointcloud = np.hstack([local_pointcloud, scan])
    local_pointcloud = local_pointcloud[:, 1:]
    local_pointcloud = np.dot(G_camera_vehicle, local_pointcloud)
    uv, depth = model.project(local_pointcloud, (height, width))
    uv = np.array(uv)
    depth = np.array(depth)
    mask = ((uv[0,:] < (width-1)) & (uv[1,:] < (height-1)) & (depth < pred_max) & (depth > pred_min))
    depth = depth[mask]
    uv = uv[:, mask]
    gt_img = np.zeros((height, width))
    coords = uv.transpose().astype(int)
    gt_img[coords[:,1],coords[:,0]] = depth    
    local_gt_depths[img_file] = gt_img    
    
    
#np.save(test_image_dir+'/vo_gt_depths.npy', vo_gt_depths)
np.save(test_image_dir+'/ins_gt_depths.npy', ins_gt_depths) 
np.save(test_image_dir+'/local_gt_depths.npy', local_gt_depths) 

('Processing:', '1417778627490633.png')
('Processing:', '1417779248343986.png')
('Processing:', '1417779808893038.png')
('Processing:', '1417777987077821.png')
('Processing:', '1417779398386141.png')
('Processing:', '1417778897766323.png')
('Processing:', '1417777907026267.png')
('Processing:', '1417779751775782.png')
('Processing:', '1417778343091903.png')
('Processing:', '1417778890329826.png')
('Processing:', '1417778842648940.png')
('Processing:', '1417779628979872.png')
('Processing:', '1417779600296312.png')
('Processing:', '1417778479260856.png')
('Processing:', '1417778128433575.png')
('Processing:', '1417778274913692.png')
('Processing:', '1417779635291545.png')
('Processing:', '1417778180426519.png')
('Processing:', '1417779114486972.png')
('Processing:', '1417779514182842.png')
('Processing:', '1417778735101028.png')
('Processing:', '1417778077815558.png')
('Processing:', '1417779581548851.png')
('Processing:', '1417778261727958.png')
('Processing:', '1417778027322445.png')


# [RobotCar] Load Ground Truth Dictionaries

In [ ]:
vo_gt_depths = np.load(test_image_dir+'/vo_gt_depths.npy').item()
ins_gt_depths = np.load(test_image_dir+'/ins_gt_depths.npy').item()
local_gt_depths = np.load(test_image_dir+'/local_gt_depths.npy').item()

# Evaluate Test Result with Ground Truth

In [ ]:
filecount = len(glob.glob1(test_image_dir,"*.png"))

vo_rms     = np.zeros(filecount, np.float32)
vo_log_rms = np.zeros(filecount, np.float32)
vo_abs_rel = np.zeros(filecount, np.float32)
vo_sq_rel  = np.zeros(filecount, np.float32)
vo_d1_all  = np.zeros(filecount, np.float32)
vo_a1      = np.zeros(filecount, np.float32)
vo_a2      = np.zeros(filecount, np.float32)
vo_a3      = np.zeros(filecount, np.float32)

ins_rms     = np.zeros(filecount, np.float32)
ins_log_rms = np.zeros(filecount, np.float32)
ins_abs_rel = np.zeros(filecount, np.float32)
ins_sq_rel  = np.zeros(filecount, np.float32)
ins_d1_all  = np.zeros(filecount, np.float32)
ins_a1      = np.zeros(filecount, np.float32)
ins_a2      = np.zeros(filecount, np.float32)
ins_a3      = np.zeros(filecount, np.float32)

local_rms     = np.zeros(filecount, np.float32)
local_log_rms = np.zeros(filecount, np.float32)
local_abs_rel = np.zeros(filecount, np.float32)
local_sq_rel  = np.zeros(filecount, np.float32)
local_d1_all  = np.zeros(filecount, np.float32)
local_a1      = np.zeros(filecount, np.float32)
local_a2      = np.zeros(filecount, np.float32)
local_a3      = np.zeros(filecount, np.float32)

# Loop trough test files
i = 0
for img_file in glob.glob1(test_image_dir,"*.png"):
    vo_abs_rel[i], vo_sq_rel[i], vo_rms[i], vo_log_rms[i], vo_a1[i], vo_a2[i], vo_a3[i] = compute_errors(vo_gt_depths[img_file][vo_gt_depths[img_file] != 0], pred_depths_dict[img_file][vo_gt_depths[img_file] != 0])
    ins_abs_rel[i], ins_sq_rel[i], ins_rms[i], ins_log_rms[i], ins_a1[i], ins_a2[i], ins_a3[i] = compute_errors(ins_gt_depths[img_file][ins_gt_depths[img_file] != 0], pred_depths_dict[img_file][ins_gt_depths[img_file] != 0])
    local_abs_rel[i], local_sq_rel[i], local_rms[i], local_log_rms[i], local_a1[i], local_a2[i], local_a3[i] = compute_errors(local_gt_depths[img_file][local_gt_depths[img_file] != 0], pred_depths_dict[img_file][local_gt_depths[img_file] != 0])
    i = i + 1

In [ ]:
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'd1_all', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(local_abs_rel.mean(), local_sq_rel.mean(), local_rms.mean(), local_log_rms.mean(), local_a1.mean(), local_a2.mean(), local_a3.mean()))

In [ ]:
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'd1_all', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(local_abs_rel.mean(), local_sq_rel.mean(), local_rms.mean(), local_log_rms.mean(), local_a1.mean(), local_a2.mean(), local_a3.mean()))

## Show Error Magnitude

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16,20))

# Sparse Map
ax = plt.subplot(3, 2, 1)
ax.set_title("Local PC - Pred Error (m) [Pred - GT]", fontsize=16)
uv, depth = model.project(local_pointcloud, image.shape)

uv = np.array(uv)
depth = np.array(depth)
mask = ((uv[0,:] < 1279) & (uv[1,:] < 959) & (depth < 80))
depth = depth[mask]
uv = uv[:, mask]

gt_img = np.zeros((960, 1280))
coords = uv.transpose().astype(int)
gt_img[coords[:,1],coords[:,0]] = depth
compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
abs_rel, sq_rel, rms, log_rms, a1, a2, a3 = compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
print("Errors for Local LIDAR:")
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))

pixels = uv.transpose()
# 1200 x 960
pred_depth_pixels = []
pred_depth_error = []
i = 0
for pixel in pixels:
    # Sparse
    pred_depth_pixels.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))])
    pred_depth_error.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))] - depth[i])
    i += 1

# Limit coordinates to image size
pred_depth_pixels = np.array(pred_depth_pixels)
pred_depth_error = np.array(pred_depth_error)
mask = (pred_depth_pixels < 80)
depth = depth[mask]
uv = uv[:, mask]
pred_depth_error = pred_depth_error[mask]
pred_depth_pixels = pred_depth_pixels[mask]
    
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=pred_depth_error,
                 edgecolors='none', norm=MidpointNormalize(midpoint=0.) , cmap='RdYlGn')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 2)
ax.set_title("Local PC - Pred Error (Rel. Mag.) [Pred - GT]", fontsize=16)
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=np.abs(pred_depth_error/depth),
                 edgecolors='none', cmap='Reds')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 3)
ax.set_title("VO PC - Pred Error (m) [Pred - GT]", fontsize=16)
uv, depth = model.project(pointcloud_vo, image.shape)

# Limit coordinates to image size
uv = np.array(uv)
depth = np.array(depth)
mask = ((uv[0,:] < 1279) & (uv[1,:] < 959) & (depth < 80))
depth = depth[mask]
uv = uv[:, mask]

gt_img = np.zeros((960, 1280))
coords = uv.transpose().astype(int)
gt_img[coords[:,1],coords[:,0]] = depth
compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
abs_rel, sq_rel, rms, log_rms, a1, a2, a3 = compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
print("Errors for INS LIDAR:")
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))

pixels = uv.transpose()
# 1200 x 960
pred_depth_pixels = []
pred_depth_error = []
i = 0
for pixel in pixels:
    # Sparse
    pred_depth_pixels.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))])
    pred_depth_error.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))] - depth[i])
    i += 1

# Limit coordinates to image size
pred_depth_pixels = np.array(pred_depth_pixels)
pred_depth_error = np.array(pred_depth_error)
mask = (pred_depth_pixels < 80)
depth = depth[mask]
uv = uv[:, mask]
pred_depth_error = pred_depth_error[mask]
pred_depth_pixels = pred_depth_pixels[mask]
    
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=pred_depth_error, norm=MidpointNormalize(midpoint=0.), cmap='RdYlGn',
                 edgecolors='none')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 4)
ax.set_title("VO PC - Pred Error (Rel. Mag.) [Pred - GT]", fontsize=16)
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=np.abs(pred_depth_error/depth),
                 edgecolors='none', cmap='Reds')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 5)
ax.set_title("INS PC - Pred Error (m) [Pred - GT]", fontsize=16)
uv, depth = model.project(pointcloud_ins, image.shape)

# Limit coordinates to image size
uv = np.array(uv)
depth = np.array(depth)
mask = ((uv[0,:] < 1279) & (uv[1,:] < 959) & (depth < 80))
depth = depth[mask]
uv = uv[:, mask]

gt_img = np.zeros((960, 1280))
coords = uv.transpose().astype(int)
gt_img[coords[:,1],coords[:,0]] = depth
compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
abs_rel, sq_rel, rms, log_rms, a1, a2, a3 = compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
print("Errors for VO LIDAR:")
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))

pixels = uv.transpose()
# 1200 x 960
pred_depth_pixels = []
pred_depth_error = []
i = 0
for pixel in pixels:
    # Sparse
    pred_depth_pixels.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))])
    pred_depth_error.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))] - depth[i])
    i += 1

# Limit coordinates to image size
pred_depth_pixels = np.array(pred_depth_pixels)
pred_depth_error = np.array(pred_depth_error)
mask = (pred_depth_pixels < 80)
depth = depth[mask]
uv = uv[:, mask]
pred_depth_error = pred_depth_error[mask]
pred_depth_pixels = pred_depth_pixels[mask]
    
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=pred_depth_error, edgecolors='none', norm=MidpointNormalize(midpoint=0.), cmap='RdYlGn')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 6)
ax.set_title("INS PC - Pred Error (Rel. Mag.) [Pred - GT]", fontsize=16)
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=np.abs(pred_depth_error/depth),
                 edgecolors='none', cmap='Reds')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

plt.savefig('result_imgs/' + str(timestamp_img) + 'results.png')